In [1]:
#defining the first line inputs
firstlines={"He saw a carrot and had to grab it;",
"He saw a sandwich and had to grab it;",
"He saw a wallet and had to grab it;",
"He saw a cabbage and had to grab it;",
"He jumped to a cabbage and had to grab it",
"He jumped to a carrot and had to grab it"
   }

PROMPT_TEMPLATES = {"Here's a rhymed poem that I know by heart:\n\n{firstline}\n",
                    "Here's a rhymed poem that I know by heart:\n{firstline}\n",
                    "A rhymed couplet:\n\n{firstline}\n",
                    "A rhymed couplet:\n{firstline}\n",
                    "Write a line that rhymes with the following line\n\n{firstline}\n",
                    "Write a line that rhymes with the following line\n{firstline}\n",
                    "Continue a rhyming poem starting with the following line:\n\n{firstline}\n",
                    "Continue a rhyming poem starting with the following line:\n{firstline}\n",
                    "Here's a rhymed poem that I know by heart:\n\n{firstline}\n",
                    "Here's a rhymed poem that I know by heart:\n{firstline}\n",
                    "Here is a rhymed poem:\n\n{firstline}\n",
                    "Here is a rhymed poem:\n{firstline}\n",
                    "Here is an example of a rhymed poem written for children:\n\n{firstline}\n",
                    "Here is an example of a rhymed poem written for children:\n{firstline}\n",
                   "This is the first line to a rhyming couplet in a poem:\n{firstline}\n\nPlease respond only with a rhymig second line that makes a nice couplet with the first line."
                   }





In [2]:
carrot_prompts = [
    PROMPT_TEMPLATE.format(firstline="He saw a carrot and had to grab it;") for PROMPT_TEMPLATE in PROMPT_TEMPLATES
]

In [3]:
carrot_prompts

['Here is a rhymed poem:\n\nHe saw a carrot and had to grab it;\n',
 'Here is an example of a rhymed poem written for children:\nHe saw a carrot and had to grab it;\n',
 'Continue a rhyming poem starting with the following line:\nHe saw a carrot and had to grab it;\n',
 'Write a line that rhymes with the following line\n\nHe saw a carrot and had to grab it;\n',
 'A rhymed couplet:\n\nHe saw a carrot and had to grab it;\n',
 'Write a line that rhymes with the following line\nHe saw a carrot and had to grab it;\n',
 'Here is an example of a rhymed poem written for children:\n\nHe saw a carrot and had to grab it;\n',
 'A rhymed couplet:\nHe saw a carrot and had to grab it;\n',
 'Continue a rhyming poem starting with the following line:\n\nHe saw a carrot and had to grab it;\n',
 "Here's a rhymed poem that I know by heart:\nHe saw a carrot and had to grab it;\n",
 "Here's a rhymed poem that I know by heart:\n\nHe saw a carrot and had to grab it;\n",
 'Here is a rhymed poem:\nHe saw a carro

In [4]:
#Gemini version, prerequisites
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 # Adjust cuXXX to your CUDA version (e.g., cu121)
!pip install transformers accelerate bitsandbytes sentencepiece huggingface_hub

Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 199.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 195.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 199.2 MB/s eta 0:00:00


In [5]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login
import os
import datetime
import gc # Garbage Collector interface

# --- Hugging Face Hub Authentication ---
# Option 1: Run this cell AFTER running `!huggingface-cli login` in a notebook terminal or separate SSH session
try:
    login() # Tries to use token saved by CLI login
    print("Hugging Face login successful (using cached token).")
except Exception as e:
    print(f"Hugging Face login failed. Ensure you have run 'huggingface-cli login' "
          f"or use Option 2. Error: {e}")

# Option 2: Paste your token directly (less secure, but convenient for notebooks)
# HUGGING_FACE_TOKEN = "YOUR_HUGGING_FACE_READ_TOKEN"
# try:
#     login(token=HUGGING_FACE_TOKEN)
#     print("Hugging Face login successful (using provided token).")
# except Exception as e:
#     print(f"Hugging Face login failed. Error: {e}")

# --- Check GPU ---
if torch.cuda.is_available():
    print(f"CUDA is available. Device: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB")
else:
    print("CUDA is not available. Running on CPU will be extremely slow.")

Hugging Face login successful (using cached token).
CUDA is available. Device: NVIDIA A100-SXM4-80GB
VRAM: 79.25 GB


In [7]:
# --- Configuration ---

# IMPORTANT: Replace placeholder Gemma 3 IDs when they become official!
MODEL_IDS = [
    "google/gemma-2-9b-it",
    "google/gemma-3-4b-it",   # Placeholder ID - Verify upon release
    "google/gemma-3-12b-it",  # Placeholder ID - Verify upon release
    "google/gemma-3-27b-it",  # Placeholder ID - Verify upon release
]

# List of prompts you want completions for
PROMPTS = [
    "The future of artificial intelligence will likely involve ",
    "Recipe for a perfect lasagna:\nIngredients:\n",
    "Explain the concept of quantum entanglement in simple terms: ",
    "Write a short story about a robot discovering music.",
]

# Generation parameters
NUM_CONTINUATIONS = 10
MAX_NEW_TOKENS = 150
TEMPERATURE = 0.7
TOP_K = 50
TOP_P = 0.95
DO_SAMPLE = True

# Output file name
TIMESTAMP = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_FILE = f"gemma_generations_notebook_{TIMESTAMP}.txt"

# --- Helper Function ---
def get_model_size_gb(model_id):
    """Estimate model size category for quantization decisions."""
    size_str = model_id.split('-')[-2] # e.g., '9b', '4b', '12b', '27b'
    if size_str.endswith('b'):
        try:
            return int(size_str[:-1])
        except ValueError:
            return 0
    return 0

print(f"Configuration set. Output will be saved to: {OUTPUT_FILE}")
# Clear file or create it for appending
with open(OUTPUT_FILE, "w") as f:
    f.write(f"Gemma Model Generations - {TIMESTAMP}\n")
    f.write("=" * 40 + "\n")

Configuration set. Output will be saved to: gemma_generations_notebook_20250416_112029.txt


In [18]:
NUM_CONTINUATIONS = 10
MAX_NEW_TOKENS = 150
TEMPERATURE = 0.7
TOP_K = 50
TOP_P = 0.95
DO_SAMPLE = True

In [6]:
PROMPTS=carrot_prompts

In [27]:
print(f"Starting generation process...")

for model_id in MODEL_IDS:
    print(f"\n--- Processing Model: {model_id} ---")

    # --- Explicitly clear memory before loading next model ---
    # This is more important in notebooks where variables might persist
    model = None
    tokenizer = None
    gc.collect() # Run Python's garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache() # Clear GPU cache

    model_size_gb = get_model_size_gb(model_id)
    load_in_4bit = False
    bnb_config = None

    # Apply quantization for larger models
    if model_size_gb > 10: # Adjust threshold based on your VRAM
        print(f"Applying 4-bit quantization for {model_size_gb}B model.")
        load_in_4bit = True
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )

    try:
        # Load tokenizer
        print("Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            print("Added EOS token as PAD token.")

        # Load model
        print("Loading model (this may take time and VRAM)...")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            quantization_config=bnb_config if load_in_4bit else None,
            trust_remote_code=True,
        )
        model.eval()
        print(f"Model {model_id} loaded successfully.")
        # Optional: print memory footprint if needed (requires accelerate > 0.20.0)
        # try:
        #     print(f"Model memory footprint: {model.get_memory_footprint() / 1e9:.2f} GB")
        # except AttributeError:
        #     print("Cannot get memory footprint (requires newer accelerate version).")


        with open(OUTPUT_FILE, "a", encoding="utf-8") as f:
            f.write(f"\n\n--- Model: {model_id} ---\n")

            for i, prompt in enumerate(PROMPTS):
                print(f"\nGenerating for Prompt {i+1}/{len(PROMPTS)}:\n{prompt}")
                f.write(f"\nPrompt {i+1}:\n{prompt}\n")
                f.write("-" * 20 + "\n")

                inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)

                print("Generating sequences...")
                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_new_tokens=MAX_NEW_TOKENS,
                        num_return_sequences=NUM_CONTINUATIONS,
                        do_sample=DO_SAMPLE,
                        temperature=TEMPERATURE,
                        top_k=TOP_K,
                        top_p=TOP_P,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                    )

                print("\nGenerated Continuations:")
                f.write("Generated Continuations:\n")
                for j, output in enumerate(outputs):
                    continuation = tokenizer.decode(
                        output[inputs.input_ids.shape[1]:],
                        skip_special_tokens=True
                    )
                    print(f"\nContinuation {j+1}:")
                    print(continuation.strip())
                    f.write(f"\nContinuation {j+1}:\n{continuation.strip()}\n")

                print("-" * 20)
                f.write("-" * 20 + "\n")

    except Exception as e:
        print(f"ERROR processing model {model_id}: {e}")
        import traceback
        traceback.print_exc() # Print full error traceback in notebook
        print("Skipping this model.")
        with open(OUTPUT_FILE, "a", encoding="utf-8") as f:
            f.write(f"\n\n--- Model: {model_id} ---")
            f.write(f"\n!!! FAILED TO LOAD OR GENERATE: {e} !!!\n")
        continue # Move to the next model

print("\n--- Generation Complete ---")
print(f"All results saved to: {OUTPUT_FILE}")

Starting generation process...

--- Processing Model: google/gemma-2-9b-it ---
Loading tokenizer...
Loading model (this may take time and VRAM)...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model google/gemma-2-9b-it loaded successfully.

Generating for Prompt 1/12:
Write a line that rhymes with the following line

He saw a carrot and had to grab it;

Generating sequences...

Generated Continuations:

Continuation 1:
He wished he'd left it in the rabbit.

Continuation 2:
He ran so fast, he nearly **splatted**.

Continuation 3:
He couldn't resist its bright **habit**. 


Let me know if you'd like to try another rhyme!

Continuation 4:
He tossed it high and **flabbergasted**.

Continuation 5:
This carrot he would surely stab it.

Continuation 6:
The phrase "had to grab it" doesn't rhyme with anything easily.  

Let's try a different approach.  We can create a rhyme by focusing on the *idea* of the line:

**Here's a line that captures the same feeling:**

He couldn't resist, that orange tabbit. 


Let me know if you'd like to explore more rhyming options!

Continuation 7:
____________________

For example:
He saw a carrot and had to grab it; **he couldn't let it sit.** 


Le

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

ERROR processing model google/gemma-3-27b-it: 
 requires the protobuf library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.

Skipping this model.

--- Generation Complete ---
All results saved to: gemma_generations_notebook_20250416_081522.txt


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py", line 2302, in _from_pretrained
    tokenizer = cls(*init_inputs, **init_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/models/gemma/tokenization_gemma.py", line 118, in __init__
    self.sp_model.Load(vocab_file)
  File "/usr/local/lib/python3.11/dist-packages/sentencepiece/__init__.py", line 961, in Load
    return self.LoadFromFile(model_file)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sentencepiece/__init__.py", line 316, in LoadFromFile
    return _sentencepiece.SentencePieceProcessor_LoadFromFile(self, arg)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: not a string

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_1343/2091065650

In [8]:
MODEL_IDS = [
    #"google/gemma-2-9b-it",
    "google/gemma-3-12b-it",
]

In [54]:

print(f"Starting generation process...")

for model_id in MODEL_IDS:
    print(f"\n--- Processing Model: {model_id} ---")

    # --- Explicitly clear memory before loading next model ---
    # This is more important in notebooks where variables might persist
    model = None
    tokenizer = None
    gc.collect() # Run Python's garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache() # Clear GPU cache

    model_size_gb = get_model_size_gb(model_id)
    load_in_4bit = False
    bnb_config = None

    # Apply quantization for larger models
    if model_size_gb > 10: # Adjust threshold based on your VRAM
        print(f"Applying 4-bit quantization for {model_size_gb}B model.")
        load_in_4bit = True
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )

    try:
        # Load tokenizer
        print("Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            print("Added EOS token as PAD token.")

        # Load model
        print("Loading model (this may take time and VRAM)...")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            quantization_config=bnb_config if load_in_4bit else None,
            trust_remote_code=True,
        )
        model.eval()
        print(f"Model {model_id} loaded successfully.")
        # Optional: print memory footprint if needed (requires accelerate > 0.20.0)
        # try:
        #     print(f"Model memory footprint: {model.get_memory_footprint() / 1e9:.2f} GB")
        # except AttributeError:
        #     print("Cannot get memory footprint (requires newer accelerate version).")


        with open(OUTPUT_FILE, "a", encoding="utf-8") as f:
            f.write(f"\n\n--- Model: {model_id} ---\n")

            for i, prompt in enumerate(PROMPTS):
                print(f"\nGenerating for Prompt {i+1}/{len(PROMPTS)}:\n{prompt}")
                f.write(f"\nPrompt {i+1}:\n{prompt}\n")
                f.write("-" * 20 + "\n")

                inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)

                print("Generating sequences...")
                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_new_tokens=MAX_NEW_TOKENS,
                        num_return_sequences=NUM_CONTINUATIONS,
                        do_sample=DO_SAMPLE,
                        temperature=TEMPERATURE,
                        top_k=TOP_K,
                        top_p=TOP_P,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                    )

                print("\nGenerated Continuations:")
                f.write("Generated Continuations:\n")
                for j, output in enumerate(outputs):
                    continuation = tokenizer.decode(
                        output[inputs.input_ids.shape[1]:],
                        skip_special_tokens=True
                    )
                    print(f"\nContinuation {j+1}:")
                    print(continuation.strip())
                    f.write(f"\nContinuation {j+1}:\n{continuation.strip()}\n")

                print("-" * 20)
                f.write("-" * 20 + "\n")

    except Exception as e:
        print(f"ERROR processing model {model_id}: {e}")
        import traceback
        traceback.print_exc() # Print full error traceback in notebook
        print("Skipping this model.")
        with open(OUTPUT_FILE, "a", encoding="utf-8") as f:
            f.write(f"\n\n--- Model: {model_id} ---")
            f.write(f"\n!!! FAILED TO LOAD OR GENERATE: {e} !!!\n")
        continue # Move to the next model

print("\n--- Generation Complete ---")
print(f"All results saved to: {OUTPUT_FILE}")

Starting generation process...

--- Processing Model: google/gemma-2-9b-it ---
Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Model google/gemma-2-9b-it loaded successfully.

Generating for Prompt 1/7:
Continue a rhyming poem starting with the following line:

He saw a carrot and had to grab it

Generating sequences...

Generated Continuations:

Continuation 1:
He saw a carrot and had to grab it,
A vibrant orange, a juicy bit.
He plucked it from the earth with glee,
And imagined all the dishes, you see.

 
He'd roast it, mash it, or perhaps make stew,
This carrot, destined for something new.
He sniffed its fragrance, sweet and bright,
A culinary masterpiece, a pure delight.



He'd sprinkle herbs and maybe some spice,
And bake it golden, oh so nice.
A simple carrot, transformed with care,
A testament to flavor beyond compare.

Continuation 2:
He saw a carrot and had to grab it,
A juicy orange treat, a vitamin hit.
He pulled it from the ground with all his might,
And took a big chomp, with pure delight. 

Now tell me more about this hungry fellow!

Continuation 3:
He was famished, his tummy felt like a rabbit


In [ ]:


MODEL_IDS = [
    "google/gemma-2-2b-it",   # Gemma 2 - 2B Instruct
    "google/gemma-3-4b-it",   # Placeholder Gemma 3 - 4B Instruct
    "google/gemma-2-9b-it",   # Gemma 2 - 9B Instruct
]

In [9]:
target_rhymes=["rabbit","habit"]

import collections
import string
# Data structure to store the analysis results
# Format: results[model_id][prompt_index] = count_of_continuations_with_rhyme
analysis_results = collections.defaultdict(dict)

# Prepare target words for efficient checking (lowercase, set)
target_rhymes_set = {word.lower() for word in target_rhymes}
# Define punctuation remover (used in helper function)
punctuation_remover = str.maketrans('', '', string.punctuation)

In [10]:
def get_model_size_gb(model_id):
    """Estimate model size category for quantization decisions."""
    parts = model_id.split('-')
    for part in reversed(parts):
        if part.endswith('b'):
            try:
                # Handle cases like '2b', '9b', '27b'
                return int(part[:-1])
            except ValueError:
                continue
    return 0 # Default if size not found

def check_continuation_for_words(continuation_text, target_word_set):
    """
    Checks if any word from target_word_set exists in the continuation_text.
    Performs case-insensitive, whole-word matching after removing punctuation.
    """
    text_lower = continuation_text.lower()
    words = text_lower.split()
    for word in words:
        # Remove punctuation from the word
        cleaned_word = word.translate(punctuation_remover)
        if cleaned_word in target_word_set:
            return True # Found a match
    return False # No match found

print("Helper functions defined.")

Helper functions defined.


In [53]:
rm -rf ~/.cache/huggingface/hub/*

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
PROMPT_TEMPLATES = {"A rhymed couplet:\n\n{firstline}\n",
                    "A rhymed couplet:\n{firstline}\n",
                    "Continue a rhyming poem starting with the following line:\n\n{firstline}\n",
                    "Continue a rhyming poem starting with the following line:\n{firstline}\n",
                    "Here is a rhymed poem:\n\n{firstline}\n",
                    "Here is a rhymed poem:\n{firstline}\n",
                    "This is the first line to a rhyming couplet in a poem:\n{firstline}\n\nPlease respond only with a rhyming second line that makes a nice couplet with the first line."
                   }

In [13]:
PROMPTS=[template.format(firstline="He saw a carrot and had to grab it") for template in PROMPT_TEMPLATES]

In [20]:
NUM_CONTINUATIONS=100

In [16]:
OUTPUT_FILE='gemma-3-12b_generations.txt'

In [ ]:
print(f"Starting generation and analysis process...")

for model_id in MODEL_IDS:
    print(f"\n--- Processing Model: {model_id} ---")

    # --- Explicitly clear memory ---
    model = None
    tokenizer = None
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    model_size_gb = get_model_size_gb(model_id)
    load_in_4bit = False
    bnb_config = None

    # Apply quantization - adjust threshold if needed based on your VRAM
    # Gemma 2B/4B might not need it, 9B might benefit on lower VRAM GPUs
    if model_size_gb > 7: # Example: Quantize models potentially larger than 7B
        print(f"Applying 4-bit quantization for {model_size_gb}B model.")
        load_in_4bit = True
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )

    try:
        # Load tokenizer
        print("Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        # Load model
        print("Loading model...")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            torch_dtype=torch.bfloat16, # Use bfloat16 if available
            quantization_config=bnb_config if load_in_4bit else None,
            trust_remote_code=True,
        )
        model.eval()
        print(f"Model {model_id} loaded successfully.")

        # Optional: Append to raw generation file
        # with open(OUTPUT_FILE, "a", encoding="utf-8") as f:
        #     f.write(f"\n\n--- Model: {model_id} ---\n")

        # Process each prompt
        for i, prompt in enumerate(PROMPTS):
            print(f"\n  Processing Prompt {i+1}/{len(PROMPTS)}...")
            print(prompt)
            # Optional: Write prompt to raw file
            with open(OUTPUT_FILE, "a", encoding="utf-8") as f:
                f.write(f"\nPrompt {i+1}:\n{prompt}\n" + "-" * 20 + "\n")

            inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)

            print(f"  Generating {NUM_CONTINUATIONS} sequences...")
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=MAX_NEW_TOKENS,
                    num_return_sequences=NUM_CONTINUATIONS,
                    do_sample=DO_SAMPLE,
                    temperature=TEMPERATURE,
                    top_k=TOP_K,
                    top_p=TOP_P,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                )

            # --- Analysis Step ---
            rhyme_continuation_count = 0
            spec_rhyme_count=collections.defaultdict(int)
            print(f"  Analyzing {NUM_CONTINUATIONS} continuations for rhymes...")
            for j, output in enumerate(outputs):
                # Decode only the generated part
                continuation = tokenizer.decode(
                    output[inputs.input_ids.shape[1]:],
                    skip_special_tokens=True
                ).strip()

                # Optional: Save raw continuation
                with open(OUTPUT_FILE, "a", encoding="utf-8") as f:
                      f.write(f"\nContinuation {j+1}:\n{continuation}\n")

                # Check if this continuation contains any target rhyme word
                if check_continuation_for_words(continuation, target_rhymes_set):
                    rhyme_continuation_count += 1
                for target in target_rhymes_set:
                    if check_continuation_for_words(continuation, {target}):
                        spec_rhyme_count[target] += 1

            # Store the result
            analysis_results[model_id][i] = rhyme_continuation_count # Store count for prompt index 'i'
            print(f"  Result for Prompt {i+1}: {rhyme_continuation_count}/{NUM_CONTINUATIONS} continuations contained a target word.")
            # Optional: Write summary to raw file
            with open(OUTPUT_FILE, "a", encoding="utf-8") as f:
                 f.write(f"\nSummary for Prompt {i+1}: {rhyme_continuation_count}/{NUM_CONTINUATIONS} contained a target word.\n")
                 for trg in target_rhymes_set: f.write(f"\nRhyme {trg}: {spec_rhyme_count[trg]}/{NUM_CONTINUATIONS} contained a target word.\n")
                 f.write("-" * 20 + "\n")

    except Exception as e:
        print(f"ERROR processing model {model_id}: {e}")
        import traceback
        traceback.print_exc()
        print("Skipping this model.")
        # Optional: Log failure to raw file
        # with open(OUTPUT_FILE, "a", encoding="utf-8") as f:
        #     f.write(f"\n\n--- Model: {model_id} ---")
        #     f.write(f"\n!!! FAILED TO LOAD OR GENERATE: {e} !!!\n")
        analysis_results[model_id] = {"ERROR": str(e)} # Mark error in results
        continue

print("\n--- Generation and Analysis Complete ---")

Starting generation and analysis process...

--- Processing Model: google/gemma-3-12b-it ---
Applying 4-bit quantization for 12B model.
Loading tokenizer...
Loading model...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Model google/gemma-3-12b-it loaded successfully.

  Processing Prompt 1/13...
Here is a rhymed poem:

He saw a carrot and had to grab it;

  Generating 100 sequences...
  Analyzing 100 continuations for rhymes...
  Result for Prompt 1: 45/100 continuations contained a target word.

  Processing Prompt 2/13...
Here is an example of a rhymed poem written for children:
He saw a carrot and had to grab it;

  Generating 100 sequences...
  Analyzing 100 continuations for rhymes...
  Result for Prompt 2: 46/100 continuations contained a target word.

  Processing Prompt 3/13...
Continue a rhyming poem starting with the following line:
He saw a carrot and had to grab it;

  Generating 100 sequences...
  Analyzing 100 continuations for rhymes...
  Result for Prompt 3: 39/100 continuations contained a target word.

  Processing Prompt 4/13...
Write a line that rhymes with the following line

He saw a carrot and had to grab it;

  Generating 100 sequences...
  Analyzing 100 continuations for rhym